# Time Series Aquire Exercises

### imports

In [44]:
import pandas as pd
import requests
import os

#### 1. Using the code from the lesson as a guide and the REST API from https://python.zach.lol/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [2]:
base_url = 'https://python.zach.lol'
print(requests.get(base_url).text)

{"api":"/api/v1","help":"/documentation"}



In [3]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



In [4]:
response = requests.get('https://python.zach.lol/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [5]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [6]:
data['payload']['items']

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'},
 {'item_brand': 'Earths Best',
  'item_id': 3,
  'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
  'item_price': 2.43,
  'item_upc12': '23923330139',
  'item_upc14': '23923330139'},
 {'item_brand': 'Boars Head',
  'item_id': 4,
  'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
  'item_price': 3.14,
  'item_upc12': '208528800007',
  'item_upc14': '208528800007'},
 {'item_brand': 'Back To Nature',
  'item_id': 5,
  'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
  'item_price': 2.61,
  'item_upc12': '759283100036',
  'item_upc14': '759283100036'},
 {'i

In [ ]:
items_list = []

response = requests.get('https://python.zach.lol/api/v1/items')
data = response.json()
n = data['payload']['max_page']

for i in range(1,n+1):
    url = 'https://python.zach.lol/api/v1/items?page='+str(i)
    response = requests.get(url)
    data = response.json()
    page_items = data['payload']['items']
    items_list.append(page_items)

In [21]:
#function to acquire items, store, and sales
def get_items(whatyouneed, cached=False):
    if cached == False:
        listy_list = []
        base_url = "https://python.zach.lol"
        api_thing = base_url + '/api/v1/'
        response = requests.get(api_thing + whatyouneed) 
        data = response.json()
        n = data['payload']['max_page']
        
        for i in range(1, n+1):
            new_url = api_thing + '?page=' + str(i)
            response = requests.get(new_url) 
            page_items = data['payload'][whatyouneed]
            listy_list += page_items
            
        whatyouneed = pd.DataFrame(listy_list)
        whatyouneed.to_csv('whatyouneed.csv')
            
    else:
        whatyouneed = pd.read_csv('whatyouneed.csv', index_col=0)
    
    return whatyouneed


In [28]:
items_df = get_items('items', cached=False)

In [29]:
items_df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [30]:
items_df.shape

(60, 6)

#### 2. Do the same thing, but for stores (https://python.zach.lol/api/v1/stores)

In [34]:
stores_df = get_items('stores', cached=False)
stores_df.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [32]:
stores_df.shape

(10, 5)

#### 3. Extract the data for sales (https://python.zach.lol/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [35]:
sales_df = get_items('sales', cached=False)

In [36]:
sales_df.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [37]:
sales_df.shape

(915000, 5)

#### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

#### 5. Combine the data from your three separate dataframes into one large dataframe.

In [38]:

df = pd.merge(sales_df, stores_df, left_on='store', right_on='store_id').drop(columns={'store'})

In [39]:
df.head()

,item,sale_amount,sale_date,sale_id,store_address,store_city,store_id,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [40]:
df = pd.merge(df, items_df, left_on='item', right_on='item_id').drop(columns={'item'})

In [41]:
df.head()

,sale_amount,sale_date,sale_id,store_address,store_city,store_id,store_state,store_zipcode,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


#### 6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [50]:
def get_germany():
    if os.path.isfile('opsd_germany_daily.csv'):
        df = pd.read_csv('opsd_germany_daily.csv', index_col=0)
    else:
        df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
        df.to_csv('opsd_germany_daily.csv')
        df.head()
    return df

In [51]:
germany = get_germany()

In [53]:
germany.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


#### 7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [ ]:
# matthews original function

def get_items(cached=False):
    if cached == False:
        items_list = []
        url = "https://python.zach.lol/api/v1/items"
        response = requests.get(url)
        data = response.json()
        n = data['payload']['max_page']
        
        for i in range(1, n+1):
            new_url = url + '?page=' + str(i)
            response = requests.get(new_url)
            data = response.json()
            page_items = data['payload']['items']
            items_list += page_items
            
        items = pd.DataFrame(items_list)
        items.to_csv('items.csv')
            
    else:
        items = pd.read_csv('items.csv', index_col=0)
    
    return items

In [54]:
#matthews rework of my function, not quite working yet
def get_whatyouneed(whatyouneed, cached=False):
    if cached == False:
        listy_list = []
        base_url = "https://python.zach.lol"
        api_thing = base_url + '/api/v1/'
        response = requests.get(api_thing + whatyouneed) 
        data = response.json()
        n = data['payload']['max_page']
        for i in range(1, n+1):
            new_url = api_thing + '?page=' + str(i)
            response = requests.get(new_url) 
            page_items = data['payload'][whatyouneed]
            listy_list += page_items
        whatyouneed = pd.DataFrame(listy_list)
        whatyouneed.to_csv(f'{whatyouneed}.csv')
    else:
        whatyouneed = pd.read_csv(f'{whatyouneed}.csv', index_col=0)
    return whatyouneed

In [55]:
itemz = get_whatyouneed('items', cached=False)
itemz.head()

OSError: [Errno 63] File name too long: '               item_brand  item_id  \\\n0                Riceland        1   \n1                  Caress        2   \n2             Earths Best        3   \n3              Boars Head        4   \n4          Back To Nature        5   \n5            Sally Hansen        6   \n6      Twinings Of London        7   \n7           Lea & Perrins        8   \n8            Van De Kamps        9   \n9                   Ahold       10   \n10             Honest Tea       11   \n11                Mueller       12   \n12  Garnier Nutritioniste       13   \n13                Pamprin       14   \n14                  Suave       15   \n15             Burts Bees       16   \n16                  Ducal       17   \n17                 Scotch       18   \n18                Careone       19   \n19           Usda Produce       20   \n20               Riceland        1   \n21                 Caress        2   \n22            Earths Best        3   \n23             Boars H